# DengAI: Predicting Disease Spread _ DRIVENDATA Competition

## 1.引入套件

In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import math

from __future__ import division
from __future__ import print_function


from sklearn import cross_validation, tree, linear_model,metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV 

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
# from xgboost import XGBRegressor,XGBClassifier

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score

import statsmodels.api as sm

from warnings import filterwarnings
filterwarnings('ignore')

## 2.觀察原始資料

In [72]:
# 將前三欄變成索引
train_features = pd.read_csv('dengue_features_train.csv',index_col=[0,1,2])
train_labels = pd.read_csv('dengue_labels_train.csv',index_col=[0,1,2])

In [73]:
print(train_features.shape)
# print(train_features.info())
train_features.head()

(1456, 22)


week_start_date   ndvi_ne   ndvi_nw   ndvi_se   ndvi_sw  \
city year weekofyear                                                           
sj   1990 18               1990/4/30  0.122600  0.103725  0.198483  0.177617   
          19                1990/5/7  0.169900  0.142175  0.162357  0.155486   
          20               1990/5/14  0.032250  0.172967  0.157200  0.170843   
          21               1990/5/21  0.128633  0.245067  0.227557  0.235886   
          22               1990/5/28  0.196200  0.262200  0.251200  0.247340   

                      precipitation_amt_mm  reanalysis_air_temp_k  \
city year weekofyear                                                
sj   1990 18                         12.42             297.572857   
          19                         22.82             298.211429   
          20                         34.54             298.781429   
          21                         15.36             298.987143   
          22                          7.52             299.518571   

                      reanalysis_avg_temp_k  reanalysis_dew_point_temp_k  \
city year weekofyear                                                       
sj   1990 18                     297.742857                   292.414286   
          19                     298.442857                   293.951429   
          20                     298.878571                   295.434286   
          21                     299.228571                   295.310000   
          22                     299.664286                   295.821429   

                      reanalysis_max_air_temp_k             ...              \
city year weekofyear                                        ...               
sj   1990 18                              299.8             ...               
          19                              300.9             ...               
          20                              300.5             ...               
          21                              301.4             ...               
          22                              301.9             ...               

                      reanalysis_relative_humidity_percent  \
city year weekofyear                                         
sj   1990 18                                     73.365714   
          19                                     77.368571   
          20                                     82.052857   
          21                                     80.337143   
          22                                     80.460000   

                      reanalysis_sat_precip_amt_mm  \
city year weekofyear                                 
sj   1990 18                                 12.42   
          19                                 22.82   
          20                                 34.54   
          21                                 15.36   
          22                                  7.52   

                      reanalysis_specific_humidity_g_per_kg  \
city year weekofyear                                          
sj   1990 18                                      14.012857   
          19                                      15.372857   
          20                                      16.848571   
          21                                      16.672857   
          22                                      17.210000   

                      reanalysis_tdtr_k  station_avg_temp_c  \
city year weekofyear                                          
sj   1990 18                   2.628571           25.442857   
          19                   2.371429           26.714286   
          20                   2.300000           26.714286   
          21                   2.428571           27.471429   
          22                   3.014286           28.942857   

                      station_diur_temp_rng_c  station_max_temp_c  \
city year weekofyear                                                
sj   1990 18                         6.900000        

In [74]:
print(train_labels.shape)
# print(train_labels.info())
train_labels.head()

(1456, 1)


total_cases
city year weekofyear             
sj   1990 18                    4
          19                    5
          20                    4
          21                    3
          22                    6

## ?:查看缺失值

In [75]:
# # 有缺失值的特徵
# print(data.isnull().any().sum(), ' / ', len(data.columns))

# # 有缺失值的資料筆數(因為有index及total_cases 所以全部皆有)
# print(data.isnull().any(axis=1).sum(), ' / ', len(data))

# # 觀察缺失比例
# total= data.isnull().sum().sort_values(ascending=False)
# percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
# missing_data = pd.concat([total, percent], axis=1, keys=['Total','Lost Percent'])
# print(data.info())
# missing_data.head()

# 聖胡安(San Juan)
位於北美洲加勒比海，為美國自治領地波多黎各的首府和最大城市，該城人口為433,733，是美國管轄的第42大城市。聖胡安是波多黎各最重要的海港以及製造業、金融、文化和旅遊中心。聖胡安都會區的人口約為200萬，占波多黎各全島人口的一半。聖胡安屬於熱帶海洋性氣候，降雨與日照高，雨季在7月至12月。

![聖胡安 (波多黎各)](imgs/San_Juan6.jpg)

# 伊基托斯(Iquitos)
位於南美洲，為秘魯亞馬遜叢林地區最大城市，人口約40萬人，為洛雷托大區省會。伊基托斯位於亞馬遜河岸邊，無公路或鐵路與外界連接，對外交通完全依靠航空和亞馬遜河航運。雖然離亞馬遜河河口有3700公里遠，但是小型海輪還是可以溯流而上抵達伊基托斯，使得該地成為世界上距離海岸最遠的海港。

![伊基托斯 (秘魯)](imgs/Iquitos.jpg)

## 3.資料前處理

In [76]:
#位於北美與南美的兩個城市應該分開來train
sj_train_features = train_features.loc['sj']
sj_train_labels = train_labels.loc['sj']
print(sj_train_features.shape)
print(sj_train_labels.shape)

iq_train_features = train_features.loc['iq']
iq_train_labels = train_labels.loc['iq']
print(iq_train_features.shape)
print(iq_train_labels.shape)

(936, 22)
(936, 1)
(520, 22)
(520, 1)


In [77]:
# # 發現1991年及1994年的 39~49週出現極端值
# sj_train_labels_90_94 = train_labels.loc['sj'].query('1990 <= year <= 1994')
# sj_train_labels_90_94__39_49 = sj_train_labels_90_94.query('39 <= weekofyear <= 49')
# sj_train_labels_90_94__39_49

# sj_train_labels_95_99 = train_labels.loc['sj'].query('1995 <= year <= 1999')
# sj_train_labels_95_99__39_49 = sj_train_labels_95_99.query('39 <= weekofyear <= 49')
# sj_train_labels_95_99__39_49

# sj_train_labels_00_04 = train_labels.loc['sj'].query('2000 <= year <= 2004')
# sj_train_labels_00_04__39_49 = sj_train_labels_00_04.query('39 <= weekofyear <= 49')
# sj_train_labels_00_04__39_49

# sj_train_labels_05_07 = train_labels.loc['sj'].query('2005 <= year <= 2007')
# sj_train_labels_05_07__39_49 = sj_train_labels_05_07.query('39 <= weekofyear <= 49')
# sj_train_labels_05_07__39_49

In [78]:
# 去除'week_start_date', 每周的起始日似乎較無意義
sj_train_features = sj_train_features.drop('week_start_date', axis=1)
iq_train_features = iq_train_features.drop('week_start_date', axis=1)
# sj_train_features
#iq_train_features

In [79]:
##################### 產生新欄位,上週的資料 ###########################
# 產生'reanalysis_specific_humidity_g_per_kg_before'欄位   (平均露點溫度)
sj_train_features['reanalysis_specific_humidity_g_per_kg_before'] = sj_train_features['reanalysis_specific_humidity_g_per_kg']
sj_train_features.reanalysis_specific_humidity_g_per_kg_before = sj_train_features.reanalysis_specific_humidity_g_per_kg.shift(1)
# print(sj_train_features.reanalysis_specific_humidity_g_per_kg_before)

iq_train_features['reanalysis_specific_humidity_g_per_kg_before'] = iq_train_features['reanalysis_specific_humidity_g_per_kg']
iq_train_features.reanalysis_specific_humidity_g_per_kg_before = iq_train_features.reanalysis_specific_humidity_g_per_kg.shift(1)
# print(iq_train_features.reanalysis_specific_humidity_g_per_kg_before)
#####################################################################################################
# 產生欄位   (平均露點溫度)
sj_train_features['reanalysis_dew_point_temp_k_before'] = sj_train_features['reanalysis_dew_point_temp_k']
sj_train_features.reanalysis_dew_point_temp_k_before = sj_train_features.reanalysis_dew_point_temp_k.shift(1)
iq_train_features['reanalysis_dew_point_temp_k_before'] = iq_train_features['reanalysis_dew_point_temp_k']
iq_train_features.reanalysis_dew_point_temp_k_before = iq_train_features.reanalysis_dew_point_temp_k.shift(1)
#####################################################################################################
# 產生欄位   (最低氣溫)
sj_train_features['reanalysis_min_air_temp_k_before'] = sj_train_features['reanalysis_min_air_temp_k']
sj_train_features.reanalysis_min_air_temp_k_before = sj_train_features.reanalysis_min_air_temp_k.shift(1)
iq_train_features['reanalysis_min_air_temp_k_before'] = iq_train_features['reanalysis_min_air_temp_k']
iq_train_features.reanalysis_min_air_temp_k_before = iq_train_features.reanalysis_min_air_temp_k.shift(1)
#####################################################################################################
# 產生欄位   (最低溫度(測站每日))
sj_train_features['station_min_temp_c_before'] = sj_train_features['station_min_temp_c']
sj_train_features.station_min_temp_c_before = sj_train_features.station_min_temp_c.shift(1)
iq_train_features['station_min_temp_c_before'] = iq_train_features['station_min_temp_c']
iq_train_features.station_min_temp_c_before = iq_train_features.station_min_temp_c.shift(1)
#####################################################################################################
# 產生欄位   (平均溫度(測站每日))
sj_train_features['station_avg_temp_c_before'] = sj_train_features['station_avg_temp_c']
sj_train_features.station_avg_temp_c_before = sj_train_features.station_avg_temp_c.shift(1)
iq_train_features['station_avg_temp_c_before'] = iq_train_features['station_avg_temp_c']
iq_train_features.station_avg_temp_c_before = iq_train_features.station_avg_temp_c.shift(1)
#####################################################################################################
# 產生欄位   (平均氣溫(Mean))
sj_train_features['reanalysis_air_temp_k_before'] = sj_train_features['reanalysis_air_temp_k']
sj_train_features.reanalysis_air_temp_k_before = sj_train_features.reanalysis_air_temp_k.shift(1)
iq_train_features['reanalysis_air_temp_k_before'] = iq_train_features['reanalysis_air_temp_k']
iq_train_features.reanalysis_air_temp_k_before = iq_train_features.reanalysis_air_temp_k.shift(1)

In [80]:
# # sj_train_features['reanalysis_min_air_temp_k'].count()   #936
# # 
# # 異常值處理(離群點)   # 好像不是用dropna?!
# a = sj_train_features['reanalysis_min_air_temp_k'] 
# sj_train_features['reanalysis_min_air_temp_k_lll'] = a - (a.std() * 3) ###### 將原始數據-3個標準差,取到極端值(回傳正數) ######
# mask = sj_train_features['reanalysis_min_air_temp_k_lll'] > 0     # 透過>0取到布林值
# sj_train_features = sj_train_features[~mask]                         # 刪除整個row
# ### sj_train_features['reanalysis_sat_precip_amt_mm'] = sj_train_features.reanalysis_sat_precip_amt_mm.[~mask]
# # sj_train_features

# # 原本927rows 刪完之後剩下906rows

In [81]:
# sj_train_features['reanalysis_min_air_temp_k'].count()

In [82]:
# sj_train_features['reanalysis_min_air_temp_k'].describe()

In [83]:
#填補缺失值
sj_train_features.fillna(method='bfill', inplace=True)
iq_train_features.fillna(method='bfill', inplace=True)

In [84]:
# 觀察'total_cases'
print('聖胡安(San Juan):')
print('病例平均數: ', sj_train_labels.mean()[0])
print('病例變異數:', sj_train_labels.var()[0])

print('\n伊基托斯(Iquitos):')
print('病例平均數: ', iq_train_labels.mean()[0])
print('病例變異數:', iq_train_labels.var()[0])

聖胡安(San Juan):
病例平均數:  34.18055555555556
病例變異數: 2640.045439691045

伊基托斯(Iquitos):
病例平均數:  7.565384615384615
病例變異數: 115.8955239365642


## 4.相關分析

In [85]:
# 將label併到feature
sj_train_features['total_cases'] = sj_train_labels.total_cases
iq_train_features['total_cases'] = iq_train_labels.total_cases

In [86]:
# 計算所有欄位(包含feature跟label)的相關係數
sj_correlations = sj_train_features.corr()
iq_correlations = iq_train_features.corr()

In [87]:
# 聖胡安(San Juan)
sj_correlations = sj_train_features.corr()
sj_correlations.loc['total_cases'].sort_values(ascending=False)

total_cases                                     1.000000
reanalysis_specific_humidity_g_per_kg_before    0.229576
reanalysis_dew_point_temp_k_before              0.225888
station_avg_temp_c_before                       0.220938
reanalysis_min_air_temp_k_before                0.211635
station_min_temp_c_before                       0.210622
reanalysis_specific_humidity_g_per_kg           0.208066
reanalysis_dew_point_temp_k                     0.204009
reanalysis_air_temp_k_before                    0.203935
station_avg_temp_c                              0.195358
reanalysis_max_air_temp_k                       0.193595
station_max_temp_c                              0.189083
reanalysis_min_air_temp_k                       0.187383
reanalysis_air_temp_k                           0.181042
station_min_temp_c                              0.176109
reanalysis_avg_temp_k                           0.174549
ndvi_ne                                         0.174359
rainingday_for_week_before     

In [88]:
# 伊基托斯(Iquitos)
iq_correlations = iq_train_features.corr()
iq_correlations.loc['total_cases'].sort_values(ascending=False)

total_cases                                     1.000000
reanalysis_specific_humidity_g_per_kg           0.235131
reanalysis_dew_point_temp_k                     0.229138
reanalysis_specific_humidity_g_per_kg_before    0.225773
reanalysis_dew_point_temp_k_before              0.220560
reanalysis_min_air_temp_k                       0.212263
station_min_temp_c_before                       0.210040
station_min_temp_c                              0.209195
reanalysis_min_air_temp_k_before                0.188151
station_avg_temp_c_before                       0.153970
station_avg_temp_c                              0.151265
reanalysis_relative_humidity_percent            0.129852
reanalysis_precip_amt_kg_per_m2                 0.101464
reanalysis_air_temp_k_before                    0.101098
reanalysis_air_temp_k                           0.095680
precipitation_amt_mm                            0.091580
reanalysis_sat_precip_amt_mm                    0.091580
reanalysis_avg_temp_k          

## 5.產生Training Data

In [89]:
# Training Data function -San Juan
def preprocess_data_sj(data_path, labels_path=None):
    #1.讀取feature 設索引
    
    df = pd.read_csv(data_path, index_col=[0, 1, 2])
        
    df['reanalysis_specific_humidity_g_per_kg_before'] = df['reanalysis_specific_humidity_g_per_kg']
    df.reanalysis_specific_humidity_g_per_kg_before = df.reanalysis_specific_humidity_g_per_kg.shift(1)  
    df['reanalysis_dew_point_temp_k_before'] = df['reanalysis_dew_point_temp_k']
    df.reanalysis_dew_point_temp_k_before = df.reanalysis_dew_point_temp_k.shift(1) 
    df['reanalysis_air_temp_k_before'] = df['reanalysis_air_temp_k']
    df.reanalysis_air_temp_k_before = df.reanalysis_air_temp_k.shift(1)  
    df['reanalysis_min_air_temp_k_before'] = df['reanalysis_min_air_temp_k']
    df.reanalysis_min_air_temp_k_before = df.reanalysis_min_air_temp_k.shift(1)  
    df['station_avg_temp_c_before'] = df['station_avg_temp_c']
    df.station_avg_temp_c_before = df.station_avg_temp_c.shift(1) 
    df['station_min_temp_c_before'] = df['station_min_temp_c']
    df.station_min_temp_c_before = df.station_min_temp_c.shift(1)  
    df['reanalysis_max_air_temp_k_before'] = df['reanalysis_max_air_temp_k']
    df.reanalysis_max_air_temp_k_before = df.reanalysis_max_air_temp_k.shift(1)
    
 # 將所有欄位都新增上週資料,取相關係數0.2以上且不重複
    features = [
#                  'reanalysis_specific_humidity_g_per_kg', 
#                  'reanalysis_dew_point_temp_k', 
#                  'station_avg_temp_c', 
#                  'reanalysis_max_air_temp_k'
                
#                 "reanalysis_air_temp_k_before",
#                 "reanalysis_specific_humidity_g_per_kg",
#                 "station_min_temp_c_before",
#                 "reanalysis_air_temp_k_before",
#                 "station_min_temp_c_before",
                "reanalysis_min_air_temp_k_before",
                "reanalysis_max_air_temp_k_before",
                "station_avg_temp_c_before",
                "reanalysis_dew_point_temp_k_before",
                "reanalysis_specific_humidity_g_per_kg_before"
               ]
    df = df[features]
    
    #3.填補缺失值
#     df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    

    #4.讀取label 設索引 
    if labels_path:
        labels = pd.read_csv(labels_path, index_col=[0, 1, 2])
        df = df.join(labels)
    
    #5.分城市 取 San Juan
    sj = df.loc['sj']
    
    return sj

In [90]:
# Training Data function -Iquitos
def preprocess_data_iq(data_path, labels_path=None):

    df = pd.read_csv(data_path, index_col=[0, 1, 2])
    
    df['reanalysis_specific_humidity_g_per_kg_before'] = df['reanalysis_specific_humidity_g_per_kg']
    df.reanalysis_specific_humidity_g_per_kg_before = df.reanalysis_specific_humidity_g_per_kg.shift(1)  
    df['reanalysis_dew_point_temp_k_before'] = df['reanalysis_dew_point_temp_k']
    df.reanalysis_dew_point_temp_k_before = df.reanalysis_dew_point_temp_k.shift(1) 
    df['reanalysis_air_temp_k_before'] = df['reanalysis_air_temp_k']
    df.reanalysis_air_temp_k_before = df.reanalysis_air_temp_k.shift(1)  
    df['reanalysis_min_air_temp_k_before'] = df['reanalysis_min_air_temp_k']
    df.reanalysis_min_air_temp_k_before = df.reanalysis_min_air_temp_k.shift(1)  
    df['station_avg_temp_c_before'] = df['station_avg_temp_c']
    df.station_avg_temp_c_before = df.station_avg_temp_c.shift(1) 
    df['station_min_temp_c_before'] = df['station_min_temp_c']
    df.station_min_temp_c_before = df.station_min_temp_c.shift(1)  
    df['reanalysis_max_air_temp_k_before'] = df['reanalysis_max_air_temp_k']
    df.reanalysis_max_air_temp_k_before = df.reanalysis_max_air_temp_k.shift(1)
    
# 將所有欄位都新增上週資料,取相關係數0.2以上且不重複
    features = [
#                  'reanalysis_specific_humidity_g_per_kg', 
#                  'reanalysis_dew_point_temp_k', 
#                  'reanalysis_min_air_temp_k', 
#                  'station_min_temp_c'
        
#                 "station_min_temp_c",
#                 "station_min_temp_c_before",
#                 "reanalysis_min_air_temp_k",
                "station_min_temp_c_before",
                "reanalysis_min_air_temp_k",
                "reanalysis_dew_point_temp_k",
                "reanalysis_specific_humidity_g_per_kg"

               ]
    df = df[features]
    

#     df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    
  
    if labels_path:
        labels = pd.read_csv(labels_path, index_col=[0, 1, 2])
        df = df.join(labels)
    
    iq = df.loc['iq']
    
    return iq

In [91]:
# 呼叫function 建立training data
sj_train = preprocess_data_sj('dengue_features_train.csv',labels_path="dengue_labels_train.csv")
iq_train = preprocess_data_iq('dengue_features_train.csv',labels_path="dengue_labels_train.csv")

# sj_train = sj_train.query('year != 1991')
# sj_train = sj_train.query('year != 1991').query('year != 1994')
# # sj_train = sj_train.query('weekofyear != 53')
# iq_train = iq_train.query('weekofyear != 1').query('weekofyear != 52').query('weekofyear != 53')

In [92]:
# 去除'total_cases'outlier
def outlierout(train):
#     train = train[np.abs(train["total_cases"]-train["total_cases"].mean())<=(7*train["total_cases"].std())] #25.3438
    train = train[np.abs(train["total_cases"]-train["total_cases"].mean())<=(6*train["total_cases"].std())] #25.2692
#     train = train[np.abs(train["total_cases"]-train["total_cases"].mean())<=(5*train["total_cases"].std())] #25.6779
    return train

sj_train = outlierout(sj_train)
iq_train = outlierout(iq_train)

In [93]:
# #取Log讓資料接近常態分配
# sj_train["total_cases"] = np.log(sj_train["total_cases"])
# iq_train["total_cases"] = np.log(iq_train["total_cases"])

# #將取完log後，total_cases產生的最大下界(-inf)轉為nan 並 去除該列
# sj_train = sj_train.replace([np.inf, -np.inf], np.nan).dropna(subset=["total_cases"], how="all")
# iq_train = iq_train.replace([np.inf, -np.inf], np.nan).dropna(subset=["total_cases"], how="all")

# print(sj_train["total_cases"].values)
# sj_train["total_cases"].shape

# #觀察 total_cases，發現資料非常態分配，透過取Log的方式讓資料接近常態分配也可讓預估上更準確
# sj_Labels = sj_train["total_cases"]
# iq_Labels = iq_train["total_cases"]
# sj_LabelsLog = np.log(sj_Labels)
# iq_LabelsLog = np.log(iq_Labels)
# # sj_Labels.head()
# # sj_LabelsLog.head()

# # sj labels
# plt.figure(figsize=(5,4))
# sns.barplot(sj_Labels.index, sj_Labels.values, alpha=0.6) 
# plt.title("sj_Labels", fontsize=16)
# plt.xlabel("sj_Labels 1990-2008", fontsize=12)
# plt.show()

# plt.figure(figsize=(5,4)) 
# sns.barplot(sj_LabelsLog.index, sj_LabelsLog.values, alpha=0.6) 
# plt.title("sj_LabelsLog", fontsize=16)
# plt.xlabel("sj_LabelsLog 1990-2008", fontsize=12)
# plt.show()

# # iq labels
# plt.figure(figsize=(5,4))
# sns.barplot(iq_Labels.index, iq_Labels.values, alpha=0.6) 
# plt.title("iq_Labels", fontsize=16)
# plt.xlabel("iq_Labels 1990-2008", fontsize=12)
# plt.show()

# plt.figure(figsize=(5,4)) 
# sns.barplot(iq_LabelsLog.index, iq_LabelsLog.values, alpha=0.6) 
# plt.title("iq_LabelsLog", fontsize=16)
# plt.xlabel("iq_LabelsLog 1990-2008", fontsize=12)
# plt.show()

In [94]:
# 查看San Juan數值資料
sj_train.describe()

,reanalysis_min_air_temp_k_before,reanalysis_max_air_temp_k_before,station_avg_temp_c_before,reanalysis_dew_point_temp_k_before,reanalysis_specific_humidity_g_per_kg_before,total_cases
count,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000
mean,297.291810,301.383836,26.986469,295.094255,16.537346,31.081897
std,1.296771,1.264526,1.420471,1.577186,1.567794,39.085912
min,292.600000,297.800000,22.842857,289.642857,11.715714,0.000000
25%,296.300000,300.400000,25.785714,293.823571,15.207500,9.000000
50%,297.500000,301.500000,27.192857,295.437143,16.820714,19.000000
75%,298.400000,302.400000,28.171429,296.420000,17.862143,37.000000
max,299.900000,304.300000,30.071429,297.795714,19.440000,333.000000


In [95]:
# 查看Iquitos數值資料
iq_train.describe()

,station_min_temp_c_before,reanalysis_min_air_temp_k,reanalysis_dew_point_temp_k,reanalysis_specific_humidity_g_per_kg,total_cases
count,518.000000,518.000000,518.000000,518.000000,518.000000
mean,21.198069,292.870270,295.494264,17.097471,7.210425
std,1.276918,1.661862,1.416757,1.445318,9.081201
min,14.700000,286.900000,290.088571,12.111429,0.000000
25%,20.600000,292.000000,294.593214,16.110000,1.000000
50%,21.300000,293.050000,295.852143,17.428571,5.000000
75%,22.000000,294.200000,296.559286,18.190000,9.000000
max,24.200000,296.000000,298.450000,20.461429,63.000000


In [96]:
# sj_train.head()

In [97]:
# iq_train.head()

## 6.資料分割 > 建模 > 訓練 > 評估 > 預測 > 輸出

## Model choice1 :SVR

In [98]:
# X = sj_train.iloc[:,0:-1].values
# y = sj_train.iloc[:,-1].values

# X_train_sj,X_test_sj,y_train_sj,y_test_sj = cross_validation.train_test_split(X,y,
#                                                                               test_size=0.3,
#                                                                               random_state=31)
# sc = StandardScaler()
# X_train_sj_std = sc.fit_transform(X_train_sj)
# X_test_sj_std = sc.transform(X_test_sj)


# from sklearn import svm, grid_search
# def svc_param_selection(X, y, nfolds):
#     Cs = [0.001, 0.01, 0.1, 1, 10]
#     gammas = [0.001, 0.01, 0.1, 1]
#     param_grid = {'C': Cs, 'gamma' : gammas}
#     grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
#     grid_search.fit(X, y)
#     grid_search.best_params_
#     return grid_search.best_params_

# print(svc_param_selection(X_train_sj_std, y_train_sj, 10))


# from sklearn.svm import SVR

# # clf_sj = SVR(C=0.001,gamma=0.1)
# clf_sj = SVR(kernel='poly', 
#             degree=5, 
#             gamma='auto',
#             coef0=0.0, 
#             tol=0.0001, 
#             C=2.0, 
#             epsilon=0.1, 
#             shrinking=True, 
#             cache_size=200, 
#             verbose=False, 
#             max_iter=-1)


# clf_sj.fit(X_train_sj_std, y_train_sj)
# y_pred_sj = clf_sj.predict(X_test_sj_std)
# print("Score:"+str(clf_sj.score(X_test_sj_std, y_test_sj)))
# print("MAE:"+str(mean_absolute_error(y_test_sj, y_pred_sj)))
# print("MSE:",metrics.mean_squared_error(y_test_sj, y_pred_sj))
# print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test_sj, y_pred_sj)))

# ###########################################################################################################################

# sj_test = preprocess_data_sj('dengue_features_test.csv')

# X2 = sj_test.values
# X2_std = sc.transform(X2)

# total_cases_sj = clf_sj.predict(X2_std).astype(int)
# print(total_cases_sj)

## Model choice2 :RandomForestRegressor

In [99]:
# # sj rfRegressor
# from sklearn.ensemble import RandomForestRegressor

# X = sj_train.iloc[:,0:-1].values
# y = sj_train.iloc[:,-1].values

# X_train_sj,X_test_sj,y_train_sj,y_test_sj = cross_validation.train_test_split(X,y,test_size=0.3,random_state=31)

# sc = StandardScaler()
# X_train_sj_std = sc.fit_transform(X_train_sj)
# X_test_sj_std = sc.transform(X_test_sj)

# rfModel_sj = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
#                                    max_features='auto', max_leaf_nodes=5,
#                                    min_impurity_decrease=0.0, min_impurity_split=None,
#                                    min_samples_leaf=3, min_samples_split=2,
#                                    min_weight_fraction_leaf=0.0,n_jobs=1,
#                                    oob_score=False, verbose=0, warm_start=False,
#                                    n_estimators=2000,random_state=31)




# rfModel_sj.fit(X_train_sj_std,y_train_sj) 
# y_pred_sj = rfModel_sj.predict(X_test_sj_std)
# print("Score:"+str(rfModel_sj.score(X_test_sj_std, y_test_sj)))
# print("MAE:"+str(mean_absolute_error(y_test_sj, y_pred_sj)))
# print("MSE:",metrics.mean_squared_error(y_test_sj, y_pred_sj))
# print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test_sj, y_pred_sj)))
# # #######################################################################
# sj_test = preprocess_data_sj('dengue_features_test.csv')

# X2 = sj_test.values
# X2_std = sc.transform(X2)

# total_cases_sj = rfModel_sj.predict(X2_std).astype(int)
# print(total_cases_sj)

In [100]:
# # iq rfRegressor
# from sklearn.ensemble import RandomForestRegressor

# X = iq_train.iloc[:,0:-1].values
# y = iq_train.iloc[:,-1].values

# X_train_iq,X_test_iq,y_train_iq,y_test_iq = cross_validation.train_test_split(X,y,test_size=0.175,random_state=201)

# sc = StandardScaler()
# X_train_iq_std = sc.fit_transform(X_train_iq)
# X_test_iq_std = sc.transform(X_test_iq)

# rfModel_iq = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
#                                    max_features='auto', max_leaf_nodes=5,
#                                    min_impurity_decrease=0.0, min_impurity_split=None,
#                                    min_samples_leaf=3, min_samples_split=2,
#                                    min_weight_fraction_leaf=0.0,n_jobs=1,
#                                    oob_score=False, verbose=0, warm_start=False,
#                                    n_estimators=2000,random_state=31)


# rfModel_iq.fit(X_train_iq_std, y_train_iq)
# y_pred_iq = rfModel_iq.predict(X_test_iq_std)
# print("Score:"+str(rfModel_iq.score(X_test_iq_std, y_test_iq)))
# print("MAE:"+str(mean_absolute_error(y_test_iq, y_pred_iq)))
# print("MSE:",metrics.mean_squared_error(y_test_iq, y_pred_iq))
# print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test_iq, y_pred_iq)))
# #######################################################################
# iq_test = preprocess_data_iq('dengue_features_test.csv')

# X2 = iq_test.values
# X2_std = sc.transform(X2)

# total_cases_iq = rfModel_iq.predict(X2_std).astype(int)
# print(total_cases_iq)

## Model choice3 :Bagging(DecisionTreeRegressor)

In [101]:
# from sklearn.ensemble import BaggingRegressor
# from sklearn.tree import DecisionTreeRegressor

# X = sj_train.iloc[:,0:-1].values
# y = sj_train.iloc[:,-1].values

# X_train_sj,X_test_sj,y_train_sj,y_test_sj = cross_validation.train_test_split(X,y,
#                                                                           test_size=0.3,
#                                                                           random_state=31)

# sc = StandardScaler()
# X_train_sj_std = sc.fit_transform(X_train_sj)
# X_test_sj_std = sc.transform(X_test_sj)

# ensemble = BaggingRegressor(base_estimator=DecisionTreeRegressor(),
#                             n_estimators=7,
#                             max_samples=1.0,
#                             max_features=1.0,
# #                             bootstrap=True,
# #                             oob_score=False,
#                             bootstrap_features=False,
#                             n_jobs=4,
#                             verbose=0,
#                             random_state=31).fit(X_train_sj_std, y_train_sj)

# ensemble.fit(X_train_sj_std,y_train_sj) 
# y_pred_sj = ensemble.predict(X_test_sj_std)
# print("Score:"+str(ensemble.score(X_test_sj_std, y_test_sj)))
# print("MAE:"+str(mean_absolute_error(y_test_sj, y_pred_sj)))
# print("MSE:",metrics.mean_squared_error(y_test_sj, y_pred_sj))
# print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test_sj, y_pred_sj)))
# # #######################################################################
# sj_test = preprocess_data_sj('dengue_features_test.csv')

# X2 = sj_test.values
# X2_std = sc.transform(X2)

# total_cases_sj = ensemble.predict(X2_std).astype(int)
# print(total_cases_sj)

## Model choice4 :XGBoost(XGBRegressor)

In [102]:
# from xgboost import XGBRegressor

# X = sj_train.iloc[:,0:-1].values
# y = sj_train.iloc[:,-1].values

# X_train_sj,X_test_sj,y_train_sj,y_test_sj = cross_validation.train_test_split(X,y,test_size=0.3,random_state=31)

# sc = StandardScaler()
# X_train_sj_std = sc.fit_transform(X_train_sj)
# X_test_sj_std = sc.transform(X_test_sj)

# xgb1 = XGBRegressor()
# parameters = {'nthread':[4, 5, 6], #when use hyperthread, xgboost may become slower
#               'objective':['reg:linear'],
#               'learning_rate': [.03, 0.05, .07], #so called `eta` value
#               'max_depth': [5, 6, 7],
#               'min_child_weight': [3, 4, 5, 6],
#               'silent': [1],
#               'subsample': [0.5],
#               'colsample_bytree': [0.5],
#               'n_estimators': [500]
#              }

# xgb_grid = GridSearchCV(xgb1,
#                         parameters,
#                         cv = 4,
#                         n_jobs = 5,
#                         verbose=True)

# xgb_grid.fit(X_train_sj_std, y_train_sj)

# print(xgb_grid.best_score_)
# print(xgb_grid.best_params_)

In [103]:
# xgb_sj = XGBRegressor(
#             nthread = 4, 
#             silent = 1,
#             max_depth = 5,
#             objective = 'reg:linear',
#             n_estimators = 55,
#             learning_rate = 0.01, 
#             n_jobs = - 1,
#             subsample = 0.9,
#             colsample_bytree = 0.6,
#             colsample_bylevel = 0.1,
#             min_child_weight = 5,
#             reg_alpha = 0.1
#           )

# xgb_sj.fit(X_train_sj_std,y_train_sj) 
# y_pred_sj = xgb_sj.predict(X_test_sj_std)
# print("Score:"+str(xgb_sj.score(X_test_sj_std, y_test_sj)))
# print("MAE:"+str(mean_absolute_error(y_test_sj, y_pred_sj)))
# print("MSE:",metrics.mean_squared_error(y_test_sj, y_pred_sj))
# print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test_sj, y_pred_sj)))
# # #######################################################################
# sj_test = preprocess_data_sj('dengue_features_test.csv')

# X2 = sj_test.values
# X2_std = sc.transform(X2)

# total_cases_sj = xgb_sj.predict(X2_std).astype(int)
# print(total_cases_sj)

## Model choice5 :Lasso

In [104]:
# ## sj Lasso
# from sklearn.linear_model import Lasso

# #25.4399
# # lasso_sj = Lasso(alpha=0.008, copy_X=True, fit_intercept=True, max_iter=1000,
# #                    normalize=False, positive=False, precompute=False, random_state=None,
# #                    selection='cyclic', tol=0.0001, warm_start=False)

# #25.4207
# # lasso_sj = Lasso(alpha=0.0008, copy_X=True, fit_intercept=True, max_iter=1000,
# #                    normalize=False, positive=False, precompute=False, random_state=None,
# #                    selection='cyclic', tol=0.0001, warm_start=False)

# # #25.3606
# # lasso_sj = Lasso(alpha=0.0008, copy_X=True, fit_intercept=True, max_iter=2000,
# #                    normalize=False, positive=False, precompute=False, random_state=None,
# #                    selection='cyclic', tol=0.0001, warm_start=False)


# lasso_sj = Lasso(alpha=0.00008, copy_X=True, fit_intercept=True, max_iter=3000,
#                    normalize=False, positive=False, precompute=False, random_state=None,
#                    selection='cyclic', tol=0.0001, warm_start=False)

# X = sj_train.iloc[:,0:-1].values
# y = sj_train.iloc[:,-1].values

# X_train_sj,X_test_sj,y_train_sj,y_test_sj = cross_validation.train_test_split(X,y,
#                                                                               test_size=0.3,
#                                                                               random_state=31)

# sc = StandardScaler()
# X_train_sj_std = sc.fit_transform(X_train_sj)
# X_test_sj_std = sc.transform(X_test_sj)

# lasso_sj.fit(X_train_sj_std, y_train_sj)
# y_pred_sj = lasso_sj.predict(X_test_sj_std)
# print("Score:"+str(lasso_sj.score(X_test_sj_std, y_test_sj)))
# print("MAE:"+str(mean_absolute_error(y_test_sj, y_pred_sj)))
# print("MSE:",metrics.mean_squared_error(y_test_sj, y_pred_sj))
# print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test_sj, y_pred_sj)))
# ########################################################################
# sj_test = preprocess_data_sj('dengue_features_test.csv')

# X2 = sj_test.values
# X2_std = sc.transform(X2)

# total_cases_sj = lasso_sj.predict(X2_std).astype(int)
# print(total_cases_sj)

In [105]:
# # iq lasso
# from sklearn.linear_model import Lasso
# # lasso_iq = Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
# #                    normalize=False, positive=False, precompute=False, random_state=None,
# #                    selection='cyclic', tol=0.0001, warm_start=False)

# lasso_iq = Lasso(alpha=0.00008, copy_X=True, fit_intercept=True, max_iter=3000,
#                    normalize=False, positive=False, precompute=False, random_state=None,
#                    selection='cyclic', tol=0.0001, warm_start=False)

# X = iq_train.iloc[:,0:-1].values
# y = iq_train.iloc[:,-1].values

# X_train_iq,X_test_iq,y_train_iq,y_test_iq = cross_validation.train_test_split(X,y,
#                                                                               test_size=0.3,
#                                                                               random_state=209)
# sc = StandardScaler()
# X_train_iq_std = sc.fit_transform(X_train_iq)
# X_test_iq_std = sc.transform(X_test_iq)

# lasso_iq.fit(X_train_iq_std, y_train_iq)
# y_pred_iq = lasso_iq.predict(X_test_iq_std)
# print("Score:"+str(lasso_iq.score(X_test_iq_std, y_test_iq)))
# print("MAE:"+str(mean_absolute_error(y_test_iq, y_pred_iq)))
# print("MSE:",metrics.mean_squared_error(y_test_iq, y_pred_iq))
# print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test_iq, y_pred_iq)))
# #######################################################################
# iq_test = preprocess_data_iq('dengue_features_test.csv')

# X2 = iq_test.values
# X2_std = sc.transform(X2)

# total_cases_iq = lasso_iq.predict(X2_std).astype(int)
# print(total_cases_iq)

## Model choice6 :Ridge

In [108]:
# sj Ridge
X = sj_train.iloc[:,0:-1].values
y = sj_train.iloc[:,-1].values

X_train_sj,X_test_sj,y_train_sj,y_test_sj = cross_validation.train_test_split(X,y,test_size=0.3,random_state=373)

sc = StandardScaler()
X_train_sj_std = sc.fit_transform(X_train_sj)
X_test_sj_std = sc.transform(X_test_sj)

from sklearn.linear_model import RidgeCV
ridgecv_sj = RidgeCV(alphas=[0.0001, 0.0001, 0.01, 0.1, 0.15, 0.155, 0.156, 0.157, 0.158, 0.159, 0.16, 0.17, 0.18, 0.19, 0.2, 0.25, 0.3])
ridgecv_sj.fit(X_train_sj_std, y_train_sj)
ridgecv_sj.alpha_  

0.1

In [109]:
from sklearn.linear_model import Ridge

#25.3317
# ridge_sj = Ridge(alpha=0.0015)

#25.3534
# ridge_sj = Ridge(alpha=0.000015, copy_X=True, fit_intercept=True, max_iter=None,
#                  normalize=False, random_state=None, solver='sag', tol=0.001)

#25.3269、25.3173、25.2692 (去除 >6倍std的outlier)
ridge_sj = Ridge(alpha=0.00015)


ridge_sj.fit(X_train_sj_std,y_train_sj) 
y_pred_sj = ridge_sj.predict(X_test_sj_std)
print("Score:"+str(ridge_sj.score(X_test_sj_std, y_test_sj)))
print("MAE:"+str(mean_absolute_error(y_test_sj, y_pred_sj)))
print("MSE:",metrics.mean_squared_error(y_test_sj, y_pred_sj))
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test_sj, y_pred_sj)))
# #######################################################################
sj_test = preprocess_data_sj('dengue_features_test.csv')

X2 = sj_test.values
X2_std = sc.transform(X2)

total_cases_sj = ridge_sj.predict(X2_std).astype(int)
# total_cases_sj = np.exp(ridge_sj.predict(X2_std)).astype(int)
print(total_cases_sj)

Score:0.05281777226703976
MAE:22.40270659119176
MSE: 1295.5474574920754
RMSE: 35.993714138611416
[23 23 23 26 21 24 26 30 36 37 38 33 35 40 37 50 48 49 54 64 60 51 47 50
 40 42 38 32 43 38 37 27 20 18 21 20 19 19 19 18 21 16 15 17 14 16 21 17
 17 19 16 15 16 18 16 24 23 40 31 31 36 33 36 39 43 44 44 41 43 46 52 45
 48 44 42 51 51 53 50 39 36 38 44 45 32 32 29 27 25 25 23 26 26 18 19 24
 24 36 28 27 32 27 26 19 32 47 40 35 39 43 63 56 49 51 55 57 60 55 55 53
 59 57 59 61 62 60 55 52 58 63 47 37 39 46 33 21 21 28 27 23 11 17 22 18
 17 12 12 12 13 12 12 17 10 16 19 11 15 14 21 15 24 37 42 42 57 51 46 46
 44 40 50 44 53 48 54 52 42 51 45 47 51 40 38 31 43 37 37 30 28 21 27 15
 15 17 15 12 10 10 12 12 11 10 10  9  9 14 16 18 17 21 24 22 35 30 21 33
 29 33 35 34 37 41 42 37 42 48 43 41 34 33 35 43 43 37 53 47 49 37 36 40
 29 24 18 17 16 14 11 11 13 13 13 11 11 14 17 18 18 15 17 17]


In [110]:
# iq Ridge
X = iq_train.iloc[:,0:-1].values
y = iq_train.iloc[:,-1].values

#25.3173、25.2692 (去除 >6倍std的outlier)
X_train_iq,X_test_iq,y_train_iq,y_test_iq = cross_validation.train_test_split(X,y,test_size=0.175,random_state=201)

sc = StandardScaler()
X_train_iq_std = sc.fit_transform(X_train_iq)
X_test_iq_std = sc.transform(X_test_iq)

from sklearn.linear_model import RidgeCV
ridgecv_iq = RidgeCV(alphas=[0.0001, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.4, 0.5, 1])
ridgecv_iq.fit(X_train_iq_std, y_train_iq)
ridgecv_iq.alpha_  

0.3

In [111]:
from sklearn.linear_model import Ridge

#25.3317
# ridge_iq = Ridge(alpha=0.27)

#25.3269
# ridge_iq = Ridge(alpha=0.275)

#25.3534
# ridge_iq = Ridge(alpha=0.276, copy_X=True, fit_intercept=True, max_iter=None,normalize=False, random_state=None, solver='sag', tol=0.001)

#25.3173
ridge_iq = Ridge(alpha=0.275)


ridge_iq.fit(X_train_iq_std,y_train_iq) 
y_pred_iq = ridge_iq.predict(X_test_iq_std)
print("Score:"+str(ridge_iq.score(X_test_iq_std, y_test_iq)))
print("MAE:"+str(mean_absolute_error(y_test_iq, y_pred_iq)))
print("MSE:",metrics.mean_squared_error(y_test_iq, y_pred_iq))
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test_iq, y_pred_iq)))
# #######################################################################
iq_test = preprocess_data_iq('dengue_features_test.csv')

X2 = iq_test.values
X2_std = sc.transform(X2)

total_cases_iq = ridge_iq.predict(X2_std).astype(int)
# total_cases_iq = np.exp(ridge_iq.predict(X2_std)).astype(int)
print(total_cases_iq)

Score:0.04044243888496579
MAE:5.965851603362371
MSE: 93.5610916316093
RMSE: 9.672698260134517
[ 7  5  7  5  0  3  5  4  5  6  6  6  7  4  6  6  7  8 12  7  7  9  9 10
  9  7  7  7  8  7  6  6  7  8  8  9  7  8  9  9  9  7  7  8 10  4  6  7
  5  3  4  2  3  4  2  3  4  4  4  3  5  1  2  6  6  7  7  8 10  8  6  8
  9  9  8 10  8  8 10  8  8  8  6  8  9  8  7  6  7  6  6  8  6  9 10  8
  8  5  5  5  5  6  3  3  4  3  3  5  0  1  2  1  3  4  4  4  5  5  3  8
  8  9  9  8 10 11 12 10 10 10  9  7  6  9  8 11  9 10  8  9 10 11 11  9
  8 10  8  3  6  8  6  7  7  4  4  4]


第266欄位 負數值補0

## Model choice7 :Liner

In [112]:
# ## model sj
# from sklearn.linear_model import LinearRegression
# regr_sj = linear_model.LinearRegression()

# X = sj_train.iloc[:,0:-1].values
# y = sj_train.iloc[:,-1].values

# X_train_sj,X_test_sj,y_train_sj,y_test_sj = cross_validation.train_test_split(X,y,
#                                                                               test_size=0.3,
#                                                                               random_state=31)
# sc = StandardScaler()
# X_train_sj_std = sc.fit_transform(X_train_sj)
# X_test_sj_std = sc.transform(X_test_sj)

# regr_sj.fit(X_train_sj_std, y_train_sj)
# y_pred_sj = regr_sj.predict(X_test_sj_std)
# print("Score:"+str(regr_sj.score(X_test_sj_std, y_test_sj)))
# print("MAE:"+str(mean_absolute_error(y_test_sj, y_pred_sj)))
# print("MSE:",metrics.mean_squared_error(y_test_sj, y_pred_sj))
# print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test_sj, y_pred_sj)))
# #######################################################################
# sj_test = preprocess_data_sj('dengue_features_test.csv')

# X2 = sj_test.values
# X2_std = sc.transform(X2)

# total_cases_sj = regr_sj.predict(X2_std).astype(int)
# print(total_cases_sj)

In [113]:
# # model iq
# from sklearn.linear_model import LinearRegression
# regr_iq = linear_model.LinearRegression()

# X = iq_train.iloc[:,0:-1].values
# y = iq_train.iloc[:,-1].values

# X_train_iq,X_test_iq,y_train_iq,y_test_iq = cross_validation.train_test_split(X,y,
#                                                                               test_size=0.3,
#                                                                               random_state=201)
# sc = StandardScaler()
# X_train_iq_std = sc.fit_transform(X_train_iq)
# X_test_iq_std = sc.transform(X_test_iq)

# regr_iq.fit(X_train_iq_std, y_train_iq)
# y_pred_iq = regr_iq.predict(X_test_iq_std)
# print("Score:"+str(regr_iq.score(X_test_iq_std, y_test_iq)))
# print("MAE:"+str(mean_absolute_error(y_test_iq, y_pred_iq)))
# print("MSE:",metrics.mean_squared_error(y_test_iq, y_pred_iq))
# print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test_iq, y_pred_iq)))
# #######################################################################
# iq_test = preprocess_data_iq('dengue_features_test.csv')

# X2 = iq_test.values
# X2_std = sc.transform(X2)

# total_cases_iq = regr_iq.predict(X2_std).astype(int)
# print(total_cases_iq)

In [114]:
# 將預測結果 numpy轉pandas
total_cases_sj =pd.DataFrame({"total_cases": total_cases_sj})
total_cases_iq =pd.DataFrame({"total_cases": total_cases_iq})
total_cases_all = pd.concat([total_cases_sj,total_cases_iq], axis=0).reset_index()    
        
#  讀取submission_format
submission_format = pd.read_csv('submission_format.csv')
submission_format = submission_format.drop(['total_cases'], axis=1)
submission_format.head()  
    
# 合併submission_format及輸出結果
submission = pd.concat([submission_format,total_cases_all['total_cases']], axis=1) 
submission  

# 輸出csv檔
submission.to_csv('output0713_1.csv', index=False)